# Introdução a Ciência dos Dados - Atividade 1


## Sumário

- [Scrapping](#Scrapping)
- [ChatGPT](#ChatGPT)

<span style="position: absolute; top: 10px; right: 10px; background: green; padding: 0.5em; color: white; border-radius: 0.25em; font-weight: bold">Vaux Gomes</span>

## Scrapping

#### Objetivo

Baixando as top 1000 músicas mais acessadas do [Letras.com](https://www.letras.com/mais-acessadas/)

#### Bibliotecas

In [29]:
# Imports
from bs4 import BeautifulSoup
from urllib.request import urlopen

import numpy as np
import pandas as pd
import json

# python3 -m pip install beautifulsoup4

#### Capturando as músicas acessadas

In [14]:
# Constants
OVERWRITE = False
MAXSONGS = 15

#
base_url = 'https://www.letras.com'
most_accessed = base_url + '/mais-acessadas/'

#
page = urlopen(most_accessed)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

#
top_musics = soup.find_all("ol", {"class": "top-list_mus"})[0].find_all('li')

#
print(f'Total de músicas: {len(top_musics)}')

Total de músicas: 1000


In [3]:
songs = []

for rank, item in enumerate(top_musics):
    songs.append({
        'rank': rank + 1,
        'singer': item.a.span.get_text(),
        'title': item.a.b.get_text(),
        'url': f'{base_url}{item.a["href"]}'
    })

# Head
songs[:3]

[{'rank': 1,
  'singer': 'Anuel AA',
  'title': 'Mejor Que Yo (part. DJ Luian y Mambo Kingz) [explícita]',
  'url': 'https://www.letras.com/anuel/mejor-que-yo-part-dj-luian-y-mambo-kingz/'},
 {'rank': 2,
  'singer': 'Grupo Frontera',
  'title': 'un x100to (part. Bad Bunny)',
  'url': 'https://www.letras.com/grupo-frontera/un-x100to-part-bad-bunny/'},
 {'rank': 3,
  'singer': 'FIFTY FIFTY',
  'title': 'Cupid (Twin Version)',
  'url': 'https://www.letras.com/fifty-fifty/cupid-twin-version/'}]

### Informações Secundárias

<span style="
    position: absolute;
    top: 20px;
    right: 10px;
    background: black;
    padding: 0.5em;
    border-radius: 0.25em;
    font-size: 14px;
    color: white;
 ">Demorado</span>

In [15]:
for i, song in enumerate(songs[:MAXSONGS]): # Update this limitation do -1
    try:
        # Loading
        page = urlopen(song['url'])
        html = page.read().decode("utf-8")
        soup = BeautifulSoup(html, "html.parser")
        
        # Data from script tag
        script = soup.find('script', {'id': 'head_scripts_vars'})
        text = script.text.split('\n')[4].strip()
        data = json.loads(text[text.find('{'):text.rfind('}') + 1])
        
        ##
        song['album'] = data['album_name'] if 'album_name' in data else ''
        song['genre'] = data['genre'] if 'genre' in data else ''
        song['lang'] = data['lyric_lang'] if 'lyric_lang' in data else ''

        # Genre (OLD)
        # song['genre'] = soup.find('div', {'id': 'breadcrumb'}) \
        #    .find_all('span', {'itemprop': 'name'})[1].get_text()
        
        # Views
        song['views'] = soup.find('div', {'class': 'cnt-info_exib'}) \
            .b.get_text().replace('.', '').strip()
        
        # Lyrics
        lyrics = soup.find('div', {'class': 'cnt-letra'})
        
        # -- Removing BR
        for br in lyrics.find_all('br'):
            br.replace_with(' ')
        
        # -- Adding space between P
        for p in lyrics.find_all('p'):
            p.replace_with(' ' + p.get_text())
            
        song['lyrics'] = lyrics.get_text().strip()
        
        #
        print(f'\r{i+1} of {len(songs)}', end='')
        
    except:
        print(f'\rERROR: {song["rank"]} - {song["title"]}')

ERROR: 560 - Blue Spring
1000 of 1000

### Salvando

In [24]:
# Converting
df = pd.DataFrame(songs)

# Parsing
df['views'] = pd.to_numeric(df['views'])

# Saving
if OVERWRITE:
    df.to_csv('../data/letras-com.csv')

---

## ChatGPT

[↑ Topo](#Sumário)

#### Objetivo

Utilizar o ChatGPT para criação de algumas _features_.

- **Atenção** Quando for usar, você deve Criar a variável de ambiente `OPENAI_KEY`

#### Loading

In [27]:
df = pd.read_csv('../data/letras-com.csv', index_col=0)
df.head()

,rank,singer,title,url,album,genre,lang,views,lyrics
0,1,Anuel AA,Mejor Que Yo (part. DJ Luian y Mambo Kingz) [e...,https://www.letras.com/anuel/mejor-que-yo-part...,NaN,reggaeton,es,114305.0,"(Real hasta la muerte, baby) (Real hasta la mu..."
1,2,Grupo Frontera,un x100to (part. Bad Bunny),https://www.letras.com/grupo-frontera/un-x100t...,NaN,regional,es,296023.0,Me queda un porciento Y lo usaré solo para dec...
2,3,FIFTY FIFTY,Cupid (Twin Version),https://www.letras.com/fifty-fifty/cupid-twin-...,The Beginning: Cupid,k-pop,en,76579.0,A hopeless romantic all my life Surrounded by ...
3,4,Eslabon Armado,Ella Baila Sola (part. Peso Pluma),https://www.letras.com/eslabon-armado/ella-bai...,Ella Baila Sola,regional,es,137514.0,"Compa, ¿qué le parece esa morra? La que anda b..."
4,5,Montesanto,Santo Es El Que Vive,https://www.letras.com/montesanto/santo-es-el-...,NaN,gospelreligioso,es,37635.0,"Santo es el que vive, santo es el que reina Sa..."


#### Bibliotecas

In [28]:
import os
import openai
import time

# python3 -m pip install openai

In [ ]:
# Authentication
openai.api_key = os.environ.get('OPENAI_KEY')

# Config
model = "text-davinci-002"
temperature = 0.5                    # Creativity level 
max_tokens = 50                      # Max answer length

---
**Exemplo de prompt**

```py
prompt = "Olá, como você está?"

# Envie a solicitação de geração de texto para o ChatGPT
response = openai.Completion.create(
    engine=model,
    prompt=prompt,
    temperature=temperature,
    max_tokens=max_tokens,
)

# Obtenha o texto gerado pelo ChatGPT a partir da resposta
generated_text = response.choices[0].text

print(generated_text)
```
---

#### Sentimentos das letras

O objetivo é criar três colunas com o sentimento principal e secundários e terciário de cada letra na visão do ChatGPT

> **ATENÇÃO** Existe um limite máximo de 60 requests por minuto. Então a rotina inclui um comando de _sleep_

<span style="
    position: absolute;
    top: 20px;
    right: 10px;
    background: black;
    padding: 0.5em;
    border-radius: 0.25em;
    font-size: 14px;
    color: white;
 ">Demorado</span>

In [ ]:
# Cleaning
sentiments = []

In [ ]:
# Prompts
for i in range(df.shape[0]):
    prompt = f'Qual é o tom emocional da seguinte letra de música "{df.lyrics[i]}"? Descreva com três palavras separadas por vírgula'
    
    response = openai.Completion.create(
        engine=model,
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    sentiments.append(response.choices[0].text)
    
    # Sleep for avoiding request rate limitations
    if i != 0 and i % 30 == 0:
        time.sleep(2)
        
    #
    print(f'\r{i+1} of {df.shape[0]}', end='')